In [8]:
import numpy as np
import cv2
import time
from IPython.display import clear_output
from tkinter import *
from pathlib import Path

def turn_to_RG (img):
    (h, w, d) = img.shape
    
    norm = np.zeros ((h, w), np.float)
    norm = img [:, :, 0].astype ('float') +\
           img [:, :, 1].astype ('float') +\
           img [:, :, 2].astype ('float')
    
    norm [norm == 0] = 5
    
    turned = np.zeros (img.shape, np.uint8)
    turned [:, :, 0] = ((img [:, :, 0].astype ('float')) / norm * 255).astype ('uint8')
    turned [:, :, 1] = ((img [:, :, 1].astype ('float')) / norm * 255).astype ('uint8')
    turned [:, :, 2] = ((img [:, :, 2].astype ('float')) / norm * 255).astype ('uint8')
    
    return turned

def find_max_bounding_box (mask):
    result = np.array (mask)
    output = cv2.connectedComponentsWithStats (mask, 8, cv2.CV_32S)
    labels_num = output      [0]
    labels     = output      [1]
    stats      = output      [2]
    sz         = stats.shape [0]
    
    max_w     = 0
    max_label = 0
    
    for label_num in range (1, sz - 1):
        if (stats [label_num, cv2.CC_STAT_AREA] > max_w):
            max_w = stats [label_num, cv2.CC_STAT_AREA]
            max_label = label_num
    
    top    = stats [max_label, cv2.CC_STAT_TOP]
    left   = stats [max_label, cv2.CC_STAT_LEFT]
    width  = stats [max_label, cv2.CC_STAT_WIDTH]
    height = stats [max_label, cv2.CC_STAT_HEIGHT]
    
    return (left, top), (left + width, top + height)

def filter_components (mask, area_low_th, area_high_th, area_rect_ratio_th, width_height_ratio_th):
    result = np.array (mask)
    output = cv2.connectedComponentsWithStats (mask, 8, cv2.CV_32S)
    labels_count = output      [0]
    labels       = output      [1]
    stats        = output      [2]
    sz           = stats.shape [0]
    
    sh = mask.shape
    
    for label_num in range (0, sz):
        width  = stats [label_num, cv2.CC_STAT_WIDTH]
        height = stats [label_num, cv2.CC_STAT_HEIGHT]
        area   = stats [label_num, cv2.CC_STAT_AREA]
        
        top    = stats [label_num, cv2.CC_STAT_TOP]
        left   = stats [label_num, cv2.CC_STAT_LEFT]
        width  = stats [label_num, cv2.CC_STAT_WIDTH]
        height = stats [label_num, cv2.CC_STAT_HEIGHT]
                
        if (area < area_low_th or
            area > area_high_th or
            width * height > area * area_rect_ratio_th or
            width / height > width_height_ratio_th or
            height / width > width_height_ratio_th or
            top == 0 or left == 0 or
            top + height == sh [1] or left + width == sh [0]):
            result [labels == label_num] = 0
    
    return result

def fill_holes (img):
    (h, w) = img.shape
    
    before_area = img.sum ()
    
    img_enlarged = np.zeros ((h + 2, w + 2), np.uint8)
    img_enlarged [1:h+1, 1:w+1] = img

    img_enl_not = cv2.bitwise_not (img_enlarged)
    th, im_th = cv2.threshold (img_enl_not, 220, 255, cv2.THRESH_BINARY_INV);

    im_floodfill = im_th.copy()

    h, w = im_th.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    cv2.floodFill(im_floodfill, mask, (0,0), 255);
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
    im_out = im_th | im_floodfill_inv
    
    result = im_out [1:h-1, 1:w-1]
    
    after_area = result.sum ()
    
    #print (before_area, after_area, before_area / after_area)
    
    #display_picture (result, str (before_area), draw = Draw_mode.PLT_WHATEVER)
    
    #if (before_area / after_area < 0.9):
    #    print ("AAAAAAAAAAAAA")
    #    result [:, :] = 0
    
    return result

CAMERA = 0
VIDEO  = 1
PHOTO  = 2

video_path = ""
video_file = ""

photo_path = "/Users/elijah/Dropbox/Programming/Warden/ellipse_detector/"

def main ():
    INPUT_SOURCE = PHOTO
    
    files = sorted (Path (photo_path).glob('**/*.png'))
    
    photo_num     = 0
    refresh_photo = True
    
    if (INPUT_SOURCE == VIDEO):
        cam = cv2.VideoCapture (video_path + video_file)

    #elif (INPUT_SOURCE == PHOTO):
    #    img = cv2.imread (photo_path + photo_file)

    #cv2.namedWindow ("frame")
    
    while (True):
        if (refresh_photo == True):
            photo_file = files [photo_num]
            img = cv2.imread (str (photo_file))
            refresh_photo = False

        if (INPUT_SOURCE == CAMERA or INPUT_SOURCE == VIDEO):
            ret, frame_ = cam.read ()

        elif (INPUT_SOURCE == PHOTO):
            frame_ = img.copy ()
        
        frame = frame_
        
        str_num = 0
        cv2.waitKey (1)    
        
        #соотношение сторон неправильное, если ресайзить так, как это здесь сейчас
        #коммент по-русски, чтобы точно не забыть
        frame_resized_ = cv2.resize (frame, (640, 480))
        
        frame_resized = frame_resized_ [100:450, 100:450, :]
        
        cv2.imshow ("frame", frame_resized)
        
        #palette obtainment
        
        ##adaptive th
        gray_frame    = cv2.cvtColor (frame_resized, cv2.COLOR_BGR2GRAY)
        
        #turned = turn_to_RG (frame_resized)
        
        blurred_frame = cv2.GaussianBlur (gray_frame, (5, 5), 0)
        _, th_frame   = cv2.threshold (blurred_frame, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        #cv2.imshow ("th_frame", th_frame)
        
        ##erode
        eroded = cv2.morphologyEx (th_frame, cv2.MORPH_ERODE, np.ones ((int (3), int (3)), np.uint8))
        #cv2.imshow ("eroded", eroded)
                
        ##chose biggest connected component
        (left, top), (right, bottom) = find_max_bounding_box (eroded)
        
        palette_cut = frame_resized [top:bottom, left:right]
        
        #cv2.imshow ("palette_cut", palette_cut)
        
        gray_palette    = cv2.cvtColor (palette_cut, cv2.COLOR_BGR2GRAY)
        blurred_palette = cv2.GaussianBlur (gray_palette, (5, 5), 0)
        _, th_palette   = cv2.threshold (blurred_palette, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        #cv2.imshow ("th_palette", th_palette)

        ##fill holes
        
        holes_filled = fill_holes (th_palette)
        #cv2.imshow ("filled", holes_filled)
        
        contours, hierarchy = cv2.findContours (th_palette, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        drawing = np.zeros_like (gray_palette)
        
        for cnt in contours:
            hull = cv2.convexHull (cnt)
            
            cv2.drawContours (drawing, [hull], 0, (255,0,255), -1)  # draw contours in red color
            
            #cv2.imshow ('output', drawing)
        
        ##show image with enlighted palette (?)
        
        #pipe obtainment
        
        palette_masked = cv2.bitwise_and (gray_palette, gray_palette, mask = drawing)
        #cv2.imshow ("masked palette", palette_masked)
        
        blurred_masked_palette = cv2.GaussianBlur (palette_masked, (5, 5), 0)
        _, th_mask_palette   = cv2.threshold (blurred_masked_palette, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        #pipes masks extraction
        
        # - invert
        # - erode/close
        # - connected components
        # - filter small
        # - filter too long
        # - enlight remaining
        
        th_mask_palette = cv2.bitwise_not (th_mask_palette)
        
        eroded_pipes = cv2.morphologyEx (th_mask_palette, cv2.MORPH_OPEN, np.ones ((int (5), int (5)), np.uint8))
        
        filtered = filter_components (eroded_pipes, 150, 3000, 3, 2)
        
        #result = palette_cut.copy ()

        result = frame_resized.copy ()
        
        print (result [left:right, top:bottom, 0].shape)
        print (filtered.shape)
        
        result [top:bottom, left:right, 0] = \
            np.add (result [top:bottom, left:right, 0],
                    np.multiply (filtered, 0.5))
        
        #steps = np.concatenate ((th_mask_palette, eroded_pipes, filtered), axis = 1)
        #cv2.imshow ("steps", steps)
        
        cv2.imshow ("result", result)
        
        #ellipse obtainment
        
#         turned = turn_to_RG (frame_resized)
#         gray = cv2.cvtColor (turned, cv2.COLOR_BGR2GRAY)
#         blur = cv2.GaussianBlur (gray, (5, 5), 0)
#         ret3, th3 = cv2.threshold (blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#         cv2.imshow ("th_", th3_)
#         cv2.imshow ("turned", turned)
        
        time.sleep (0.02)

        clear_output (wait=True)
        
        keyb = cv2.waitKey (1) & 0xFF
        
        if (keyb == ord('n')):
            photo_num += 1
            
            if (photo_num >= len (files)):
                photo_num = 0
            
            refresh_photo = True

        if (keyb == ord('p')):
            photo_num -= 1
            
            if (photo_num < 0):
                photo_num = len (files) - 1
            
            refresh_photo = True

        if (keyb == ord('q')):
            break

    #cam.release ()

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main ()

(227, 218)
(218, 227)
